In [5]:
#本文档进行预测处理
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from huawei.split_data import get_data
from huawei.state_predict import mean_value, mv_and_ar,arma
import datetime

##设置绘图的大小
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2

In [41]:
##读入数据
# 导入数据
#选用的服务器类型
columns = ["id","ecs","date","time"]
#读取数据
data = pd.read_table('data_set/data_2015_12_to_2016_01.txt',header=None,delim_whitespace=True,names=columns)
#转化成日期格式
data['date'] = pd.to_datetime(data['date'])
####设置需要的日期
train_begin_date = datetime.datetime(2015, 12, 1)
train_end_date = datetime.datetime(2016, 1, 22)
test_begin_date = datetime.datetime(2016, 1, 23)
test_end_date = datetime.datetime(2016, 1, 30)
train_data, test_data, test_target = get_data(data,train_begin_date,train_end_date,test_begin_date,test_end_date)
predict_len = int(str(test_end_date - test_begin_date)[0:2])

#print(train_data)
#print(test_data)

In [71]:
###对train_data进行处理
ecs1_data = train_data["flavor1"]
split_windows = 7
ecs1_data = pd.DataFrame(ecs1_data)
#ecs1_data.rename(index=str, columns={"flavor1": "var(t)"})
cols, names = list(), list()
for i in range(split_windows+1):
    cols.append(ecs1_data.shift(i))
    names += [('var(t-%d)' % (i))]
agg = pd.concat(cols, axis=1)
agg.columns = names
agg.dropna(inplace=True)
agg = agg.rename(index=str, columns={"var(t-0)": "target"})
target = agg["target"]
agg.drop('target',axis=1, inplace=True)
index = [i for i in range(len(target)-split_windows, len(target))]
need_data = target[index].values
print (need_data)

[  3.   1.   3.   3.   2.   5.  10.]


In [72]:
from huawei.split_data import series_to_supervised
from sklearn import linear_model
model_LinearRegression = linear_model.LinearRegression()
ecs1_data = train_data["flavor1"]
split_windows = 7
train,target = series_to_supervised(ecs1_data,split_windows)
model_LinearRegression.fit(train,target)
need_data= [3.0,1.0,3.0,3.0,2.0,5.0,10.0]
model_LinearRegression.predict(need_data)

C:\Users\cao\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 5.21443874])